# CD4+ T cell behavior in COVID-19 infection


* IFNG_1: small effect, may activate some anergic->Th1, in CoV-TGFB may change Th17->Th1R
* IL21_1: no, secondary effects are strong inflamation/anergy
* IL10_0: no, some Th1R->Th1 but also induces anergy, in CoV-TGFB may change Th1R->Th17 
* TGFB_0: cautiously, good in CoV-Mod, small in CoV-Sev, may induce Th17 in CoV-TGFB so neccesary sustained treatment, see mutants to avoid CoV-TGFB, 
* SOCS_1: yes, increases Th1 with little secondary effects, see mutants, it increases Th1-basin, small effec in Th17-Th1R in CoV-TGFB
* SOCS_3: small effect, may induce anergy


In [1]:
import pandas as pd

env_cov = ['covmod', 'covsev', 'covtgfb', 'covil10', 'covlev']
ana_cov = ['cfm','mut']

data = []
for ana in ana_cov:
    for env in env_cov:
        filename = 'results/thsocs_{}_{}.csv'.format(ana,env)
        print(filename)
        df = pd.read_csv(filename)
        df['analysis'] = ana
        df['environment'] = env
        data.append(df)
data = pd.concat(data)
data.tail()

results/thsocs_cfm_covmod.csv
results/thsocs_cfm_covsev.csv
results/thsocs_cfm_covtgfb.csv
results/thsocs_cfm_covil10.csv
results/thsocs_cfm_covlev.csv
results/thsocs_mut_covmod.csv
results/thsocs_mut_covsev.csv
results/thsocs_mut_covtgfb.csv
results/thsocs_mut_covil10.csv
results/thsocs_mut_covlev.csv


,initial,final,genes,values,analysis,environment,freq
74,Treg,Tr,IL2,0,mut,covlev,1.0
75,Treg,Tr,IL2e,0,mut,covlev,1.0
76,Treg,Tr,IL6,1,mut,covlev,1.0
77,Treg,Tr*,RORGT,1,mut,covlev,1.0
78,Treg,Tr*,TGFB,0,mut,covlev,1.0


In [2]:

#diff =  data[ (data['initial']!=data['final']) ]
diff = pd.DataFrame()
diff['Intervention'] = data['genes'] + '=' + data['values'].astype(str)
diff['Transition'] = data['initial'] + '→' + data['final']
diff['Perturb time'] = data['analysis'].replace({'cfm':'t=1','mut':'t=∞'})
diff['Environment'] = data['environment']
diff['Filter'] = data['final'].map({'Th1':'ΔTh1'}).fillna('Other')
diff = diff[data['initial']!=data['final']].drop_duplicates()
diff.tail()

,Intervention,Transition,Perturb time,Environment,Filter
74,IL2=0,Treg→Tr,t=∞,covlev,Other
75,IL2e=0,Treg→Tr,t=∞,covlev,Other
76,IL6=1,Treg→Tr,t=∞,covlev,Other
77,RORGT=1,Treg→Tr*,t=∞,covlev,Other
78,TGFB=0,Treg→Tr*,t=∞,covlev,Other


## Intervention on extrinsic cytokines

In [3]:
diff[(diff['Intervention'].str.contains('e')) & (diff['Environment']=='covmod')]


,Intervention,Transition,Perturb time,Environment,Filter
64,IL10e=1,Tex→Th1,t=1,covmod,ΔTh1
80,TGFBe=1,Tex→Th1R,t=1,covmod,Other
83,IL10e=1,Tex→Th2R,t=1,covmod,Other
120,IL10e=1,Th1→Th1R,t=1,covmod,Other
124,TGFBe=1,Th1→Th1R,t=1,covmod,Other
271,TGFBe=1,Th2R→Tr,t=1,covmod,Other
1,IFNGe=0,Naive→Tex,t=∞,covmod,Other
2,IL10e=1,Naive→Tex,t=∞,covmod,Other
3,IL12e=0,Naive→Tex,t=∞,covmod,Other
5,IL2e=1,Naive→Tex,t=∞,covmod,Other


In [4]:
diff[(diff['Intervention'].str.contains('e')) & (diff['Environment']!='covmod')]


,Intervention,Transition,Perturb time,Environment,Filter
83,IL10e=1,Tex→Th2R,t=1,covsev,Other
120,IL10e=1,Th1→Th1R,t=1,covsev,Other
124,TGFBe=1,Th1→Th1R,t=1,covsev,Other
129,IFNGe=0,Th1R→Tex,t=1,covsev,Other
271,TGFBe=1,Th2R→Tr,t=1,covsev,Other
...,...,...,...,...,...
28,IL10e=1,Th1→Th1R,t=∞,covlev,Other
30,TGFBe=1,Th1→Th1R,t=∞,covlev,Other
53,TGFBe=1,Th2R→Tr,t=∞,covlev,Other
57,TGFBe=1,Th2R→Treg,t=∞,covlev,Other


## Interventions that lead towards Th1

Ignore covmod, focus in severe covid

In [5]:
interventions =  diff[(diff['Filter']=='ΔTh1') ]
display(interventions)
interventions = interventions[(interventions['Environment']!='covmod')]
interventions = interventions.drop_duplicates()

,Intervention,Transition,Perturb time,Environment,Filter
60,IFNG=1,Tex→Th1,t=1,covmod,ΔTh1
63,IL10=1,Tex→Th1,t=1,covmod,ΔTh1
64,IL10e=1,Tex→Th1,t=1,covmod,ΔTh1
65,IL2=0,Tex→Th1,t=1,covmod,ΔTh1
67,IL4=1,Tex→Th1,t=1,covmod,ΔTh1
...,...,...,...,...,...
34,IL10=0,Th1R→Th1,t=∞,covlev,ΔTh1
35,TGFB=0,Th1R→Th1,t=∞,covlev,ΔTh1
48,GATA3=0,Th2R→Th1,t=∞,covlev,ΔTh1
49,TBET=1,Th2R→Th1,t=∞,covlev,ΔTh1


In [6]:
interventions = interventions['Intervention'].unique()
interventions

array(['IFNG=1', 'SOCS1=0', 'IL10=0', 'IL6=1', 'SOCS3=0', 'TGFB=0',
       'IL2=0', 'IL6=0', 'GATA3=0', 'TBET=1'], dtype=object)

## Secondary effects

In [7]:
effect = diff[(diff['Intervention'].isin(interventions))]
effect = pd.pivot_table(effect, index=['Intervention','Filter'],columns=['Environment','Perturb time'],
                        values='Transition', aggfunc=lambda x:'\n'.join(x), fill_value='-')
effect

Environment                            covil10                                 \
Perturb time                               t=1                            t=∞   
Intervention Filter                                                             
GATA3=0      Other                     Th2R→Tr                        Th2R→Tr   
             ΔTh1                            -                              -   
IFNG=1       Other          Th2R→Th1R\nTr→Th1R  Naive→Tex\nTh2R→Th1R\nTr→Th1R   
             ΔTh1                            -                              -   
IL10=0       Other   Th1R→Tr\nTh2R→Tr\nTr→Th1R     Th1R→Tex\nTh2R→Tex\nTr→Tex   
             ΔTh1                            -                       Th1R→Th1   
IL2=0        Other                           -                              -   
             ΔTh1                            -                              -   
IL6=0        Other                           -                              -   
             ΔTh1                            -                              -   
IL6=1        Other            Th1R→Tr\nTh2R→Tr    Naive→Tex\nTh1R→Tr\nTh2R→Tr   
             ΔTh1                            -                              -   
SOCS1=0      Other                           -                      Naive→Tex   
             ΔTh1                            -                              -   
SOCS3=0      Other            Th1R→Tr\nTh2R→Tr               Th1R→Tr\nTh2R→Tr   
             ΔTh1                            -                              -   
TBET=1       Other          Th2R→Th1R\nTr→Th1R  Naive→Tex\nTh2R→Th1R\nTr→Th1R   
             ΔTh1                            -                              -   
TGFB=0       Other                           -                              -   
             ΔTh1                            -                              -   

Environment                                                covlev  \
Perturb time                                                  t=1   
Intervention Filter                                                 
GATA3=0      Other                                       Th2R→Tex   
             ΔTh1                                        Th2R→Th1   
IFNG=1       Other                  Th2R→Th1R\nTr→Th1R\nTreg→Th1R   
             ΔTh1                                         Tex→Th1   
IL10=0       Other                   Th2R→Tex\nTr→Th1R\nTreg→Th1R   
             ΔTh1                                        Th1R→Th1   
IL2=0        Other                                        Treg→Tr   
             ΔTh1                                         Tex→Th1   
IL6=0        Other                                              -   
             ΔTh1                                         Tex→Th1   
IL6=1        Other   Th1→Tex\nTh1R→Tex\nTh1R→Tr\nTh2R→Tr\nTreg→Tr   
             ΔTh1                                               -   
SOCS1=0      Other                                              -   
             ΔTh1                                         Tex→Th1   
SOCS3=0      Other                                              -   
             ΔTh1                                               -   
TBET=1       Other                   Th2R→Tex\nTr→Th1R\nTreg→Th1R   
             ΔTh1                                        Th2R→Th1   
TGFB=0       Other                             Th1R→Tr*\nTreg→Tr*   
             ΔTh1                                Th1R→Th1\nTr→Th1   

Environment                                                        \
Perturb time                                                  t=∞   
Intervention Filter                                                 
GATA3=0      Other                                       Th2R→Tex   
             ΔTh1                                        Th2R→Th1   
IFNG=1       Other                  Th2R→Th1R\nTr→Th1R\nTreg→Th1R   
             ΔTh1                                         Tex→Th1   
IL10=0       Other                   Th2R→Tex\nTr→Th1R\nTreg→Th1R

In [8]:
effect.to_csv('results/thsocs_interventions.csv')

## Th1/Th1R transition rate

In [9]:
count = data[['environment','final','values']]
#count.groupby(['environment','final']).count()
count = pd.pivot_table(count, index='environment',columns='final', 
                       aggfunc='count')
count.columns = count.columns.get_level_values(1)
count

final,Naive,NoLabel,Tex,Th1,Th17,Th17R,Th1R,Th2,Th2R,Tr,Tr*,Treg
environment,,,,,,,,,,,,
covil10,21.0,NaN,40.0,2.0,NaN,NaN,200.0,NaN,65.0,158.0,12.0,6.0
covlev,NaN,2.0,32.0,18.0,6.0,NaN,34.0,4.0,10.0,24.0,12.0,16.0
covmod,21.0,NaN,125.0,104.0,3.0,NaN,206.0,3.0,54.0,58.0,10.0,4.0
covsev,21.0,NaN,183.0,64.0,7.0,NaN,175.0,4.0,54.0,64.0,12.0,4.0
covtgfb,21.0,NaN,26.0,1.0,39.0,3.0,99.0,NaN,5.0,50.0,5.0,3.0


In [10]:
count['Th1']/count['Th1R']

environment
covil10    0.010000
covlev     0.529412
covmod     0.504854
covsev     0.365714
covtgfb    0.010101
dtype: float64

In [14]:
count['Th17']/(count['Tr']+count['Tr*']+count['Treg'])

environment
covil10         NaN
covlev     0.115385
covmod     0.041667
covsev     0.087500
covtgfb    0.672414
dtype: float64

In [15]:
count['Th17']/count['Th1']

environment
covil10          NaN
covlev      0.333333
covmod      0.028846
covsev      0.109375
covtgfb    39.000000
dtype: float64

In [17]:
count.sum(axis=1)


environment
covil10    504.0
covlev     158.0
covmod     588.0
covsev     588.0
covtgfb    252.0
dtype: float64